## Config

In [1]:
# change to raw ../../data file
INPUT_DATA_PATH = "../../data/raw/Bevolkingsontwikkeling__regio_per_maand_12122024_160930.csv"
INPUT_SHAPEFILE_PATH = "../../data/gemeente_shapefiles/gemeente_shapefile.shp"

# set true or false
SAVE_OUTPUT = True
# change to filename
OUTPUT_DATA_PATH = "../../data/processed/bevolkingsontwikkeling.csv"

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd

## Load data

In [4]:
# load data
df = pd.read_csv(INPUT_DATA_PATH, sep = ';')
gdf = gpd.read_file(INPUT_SHAPEFILE_PATH, sep = ';')

In [5]:
gdf = gdf.to_crs("EPSG:4326")

In [6]:
gdf

,gm_code,gm_naam,geometry
0,GM0003,Appingedam,"POLYGON ((6.80583 53.31703, 6.80588 53.31704, ..."
1,GM0010,Delfzijl,"MULTIPOLYGON (((7.07833 53.26658, 7.07826 53.2..."
2,GM0014,Groningen,"POLYGON ((6.48047 53.24551, 6.48050 53.24579, ..."
3,GM0024,Loppersum,"POLYGON ((6.73921 53.38812, 6.73925 53.38815, ..."
4,GM0034,Almere,"MULTIPOLYGON (((5.21106 52.33159, 5.21106 52.3..."
...,...,...,...
350,GM1963,Hoeksche Waard,"MULTIPOLYGON (((4.57231 51.70007, 4.57053 51.6..."
351,GM1966,Het Hogeland,"MULTIPOLYGON (((6.29052 53.33942, 6.29052 53.3..."
352,GM1969,Westerkwartier,"POLYGON ((6.23175 53.25595, 6.23176 53.25599, ..."
353,GM1970,Noardeast-Fryslân,"MULTIPOLYGON (((6.17582 53.34599, 6.17582 53.3..."


## Data management

In [7]:
df.head()

,Perioden,Regio's,Vestiging in de gemeente /Vestiging vanuit een andere gemeente (aantal),Vestiging in de gemeente /Immigratie (aantal),Vertrek uit de gemeente/Vertrek naar andere gemeente (aantal),Vertrek uit de gemeente/Emigratie inclusief adm. correcties (aantal)
0,2002,Aa en Hunze,1121.0,232.0,1498.0,119.0
1,2003,Aa en Hunze,1071.0,56.0,1111.0,153.0
2,2004,Aa en Hunze,1104.0,63.0,1023.0,54.0
3,2005,Aa en Hunze,1254.0,68.0,1062.0,81.0
4,2006,Aa en Hunze,1320.0,49.0,1222.0,104.0


In [8]:
df

,Perioden,Regio's,Vestiging in de gemeente /Vestiging vanuit een andere gemeente (aantal),Vestiging in de gemeente /Immigratie (aantal),Vertrek uit de gemeente/Vertrek naar andere gemeente (aantal),Vertrek uit de gemeente/Emigratie inclusief adm. correcties (aantal)
0,2002,Aa en Hunze,1121.0,232.0,1498.0,119.0
1,2003,Aa en Hunze,1071.0,56.0,1111.0,153.0
2,2004,Aa en Hunze,1104.0,63.0,1023.0,54.0
3,2005,Aa en Hunze,1254.0,68.0,1062.0,81.0
4,2006,Aa en Hunze,1320.0,49.0,1222.0,104.0
...,...,...,...,...,...,...
12227,2019,Zwolle,5940.0,975.0,5356.0,715.0
12228,2020,Zwolle,6022.0,686.0,5574.0,562.0
12229,2021,Zwolle,5714.0,843.0,5554.0,558.0
12230,2022,Zwolle,5775.0,1947.0,5401.0,913.0


In [9]:
columns_to_pivot = [
    "Vestiging in de gemeente /Vestiging vanuit een andere gemeente (aantal)",
    "Vestiging in de gemeente /Immigratie (aantal)",
    "Vertrek uit de gemeente/Vertrek naar andere gemeente (aantal)",
    "Vertrek uit de gemeente/Emigratie inclusief adm. correcties (aantal)"
]

new_dataframes = {}

# Itereer door elke kolom en maak een pivot
for col in columns_to_pivot:
    df_pivot = df.pivot(index="Regio's", columns="Perioden", values=col).reset_index()
    new_dataframes[col] = df_pivot  # Sla de nieuwe dataframe op

# Toegang tot de individuele dataframes
vestiging_gm = new_dataframes[columns_to_pivot[0]]
immigratie = new_dataframes[columns_to_pivot[1]]
vertek_gm = new_dataframes[columns_to_pivot[2]]
emigratie = new_dataframes[columns_to_pivot[3]]

In [10]:
dfs = [vestiging_gm, immigratie, vertek_gm, emigratie]

In [11]:
mapping = {
    "Beek (L.)": "Beek",
    "'s-Gravenhage (gemeente)": "'s-Gravenhage",
    "Groningen (gemeente)": "Groningen",
    "Hengelo (O.)": "Hengelo",
    "Laren (NH.)": "Laren",
    "Middelburg (Z.)": "Middelburg",
    "Rijswijk (ZH.)": "Rijswijk",
    "Stein (L.)": "Stein",
    "Utrecht (gemeente)": "Utrecht",
}

# Functie om de waarde aan te passen met de mapping
def update_name(name):
    return mapping.get(name, name)  # Gebruik de waarde uit de mapping, of laat ongewijzigd als niet gevonden


In [12]:
for i in dfs:
  i.rename(columns={"Regio's" : "gm_naam"}, inplace =True)


In [13]:
# Pas de kolom aan
for i in dfs:
    i["gm_naam"] = i["gm_naam"].apply(update_name)


In [14]:
vestiging_gm = vestiging_gm.merge(gdf, on = "gm_naam", how='right')
immigratie = immigratie.merge(gdf, on = "gm_naam", how='right')
vertek_gm = vertek_gm.merge(gdf, on = "gm_naam", how='right')
emigratie = emigratie.merge(gdf, on = "gm_naam", how='right')

## Save output

In [17]:
if SAVE_OUTPUT:
    vestiging_gm.to_csv("../../data/processed/vestiging_gm.csv", sep = ';', index = False)

if SAVE_OUTPUT:
    immigratie.to_csv("../../data/processed/immigratie.csv", sep = ';', index = False)

if SAVE_OUTPUT:
    vertek_gm.to_csv("../../data/processed/vertek_gm.csv", sep = ';', index = False)

if SAVE_OUTPUT:
    emigratie.to_csv("../../data/processed/emigratie.csv", sep = ';', index = False)

## Watermark

In [18]:
!python -m pip install watermark --quiet

In [19]:
%load_ext watermark

In [20]:
%watermark

Last updated: 2025-01-27T10:49:24.549353+01:00

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.17.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [21]:
%watermark --iversions

geopandas: 0.14.0
numpy    : 1.26.1
pandas   : 2.1.2



In [22]:
!lsb_release -a

'lsb_release' is not recognized as an internal or external command,
operable program or batch file.
